In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
df = pd.read_csv('data/shared_articles.csv')
df =df[df['eventType'] == 'CONTENT SHARED']
df.shape

(3047, 13)

In [5]:
df1 = pd.read_csv('data/users_interactions.csv')
event_type_strength = {
    'VIEW': 1.0,
    'LIKE': 2.0, 
    'BOOKMARK': 2.5, 
    'FOLLOW': 3.0,
    'COMMENT CREATED': 4.0,  
}
 
df1['eventStrength'] = df1.eventType.apply(lambda x: event_type_strength[x])
df1['eventStrength'].mean()

1.2362885828078327

In [6]:
users_df = (
    df1
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
 
users_5_df = users_df[users_df >= 5].reset_index()[['personId']]
print(len(users_5_df))

1140


In [7]:
int_from_users5_df = df1.loc[np.in1d(df1.personId,
            users_5_df)] #этот метод используется, если мы хотим найти  пересечение двух одномерных массивов NumPy
print(int_from_users5_df.shape)

(69868, 9)


In [8]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

full_df = (
    int_from_users5_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
full_df['last_timestamp'] = (
    int_from_users5_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
full_df['last_timestamp'].mean()

1470605340.0403006

In [9]:
split_ts = 1475519545
train_df = full_df.loc[full_df.last_timestamp < split_ts].copy()
test_df = full_df.loc[full_df.last_timestamp >= split_ts].copy()
 
print(len(train_df))

29325


In [10]:
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]

In [11]:
popular = (
    train_df
    .groupby('contentId')
    .eventStrength.sum().reset_index()
    .sort_values('eventStrength', ascending=False)
    .contentId.values
)
popular[0]

-6783772548752091658

In [12]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
calc_precision('popular')

0.006454207722621089